In [12]:
%load_ext blackcellmagic
%load_ext autoreload
%autoreload 2

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import os
os.environ["DATA_DIR"]="/home/pacorofe/Dropbox (CRG ADV)/ZFISH_MICs/_BSocial/2022_ANALYSIS_social"

from ipywidgets import interact
from confapp import conf
import pandas as pd
import numpy as np

from PIL import Image
from IPython.display import display
from natsort import natsorted

from mic_analysis.plot_interactive import display_partition_figures, get_possible_variables, get_possible_outliers, visualize_outlier, visualize_video

In [14]:
print(os.environ["DATA_DIR"])

/home/pacorofe/Dropbox (CRG ADV)/ZFISH_MICs/_BSocial/2022_ANALYSIS_social


In [15]:
DATE = "20220727"
STAT = "mean"

# Visualize pre-plotted figures

In [16]:
videos_table = pd.read_csv(conf.VIDEOS_INDEX_FILE_NAME)

partition_col = "line_experiment"
video_uid_col = "trial_uid"


possible_partition_uids = natsorted(videos_table[partition_col].unique())

save_path = os.path.join(conf.GENERATED_FIGURES_PATH, f"summary_{partition_col}_{DATE}_{STAT}")
if not os.path.isdir(save_path):
    os.makedirs(save_path)
    
per_video_load_path = os.path.join(conf.GENERATED_FIGURES_PATH, f"summary_per_video_{partition_col}_{DATE}")


@interact(
    partition_uid=possible_partition_uids,
    var_type=["indiv", "group", "indiv_nb"],
    visualize_outliers=False,
    visualize_videos=False,
)
def summary_partition(
    partition_uid="ap1g1_1_1", 
    var_type="indiv", 
):

    load_path = os.path.join(save_path, partition_uid)
    videos_summary_figs_path = os.path.join(per_video_load_path, f"{partition_uid}")
    
    display_partition_figures(load_path, var_type)
    
    
    
    partition_videos = videos_table[(videos_table[partition_col] == partition_uid) & videos_table.valid_for_analysis]
    genotype_groups = partition_videos.genotype_group.unique()
    @interact(genotype_group=genotype_groups)
    def select_genotype(genotype_group):
        genotype_videos = partition_videos[partition_videos.genotype_group == genotype_group]
        partition_genotype_videos_uids = genotype_videos.trial_uid.unique()
        
        @interact(video_uid=partition_genotype_videos_uids)
        def show_video(video_uid):
            visualize_video(
                videos_summary_figs_path, videos_table, video_uid_col, video_uid, var_type
            )

            outliers_path = os.path.join(load_path, "all_outliers.csv")
            if os.path.isfile(outliers_path):
                outliers = pd.read_csv(outliers_path)
                outliers_per_variable_type = outliers[outliers.var_type == var_type]
                variables = get_possible_variables(outliers_per_variable_type)
                

                @interact(variable=variables)
                def show_variable(variable):
                    outliers_uids = get_possible_outliers(outliers_per_variable_type, variable, video_uid_col)

                    @interact(outlier=outliers_uids)
                    def plot_outlier_var(outlier):
                        visualize_outlier(
                            outliers_per_variable_type, outlier, videos_summary_figs_path, variable, video_uid_col
                        )


interactive(children=(Dropdown(description='partition_uid', options=('ap1g1_1_1', 'apbb1_1_1', 'asap1b_7_1', '…